In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time
import os

import re

from bs4 import BeautifulSoup

In [2]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(60)

wait = WebDriverWait(driver, 30)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

def sync_get_elements_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_elements_by_xpath(xpath)

In [4]:
def sign_in():
    email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
    password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
    sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    sign_in_btn.click()


In [5]:
base = 'https://classroom.udacity.com/nanodegrees/nd101/parts/94643112-2cab-46f8-a5be-1b6e4fa7a211/modules/89a1ec1d-4c22-4a77-b230-b0da99240c89/lessons/07f472eb-0210-446f-8ec2-d297b06c86d0/concepts/7d480208-0453-4457-97c3-56c720c23a89'

In [6]:
driver.get(base)
sign_in()
time.sleep(5)

In [7]:
html_src = driver.page_source
# print(html_src)

Use regex to find css stylesheet tags

In [8]:
matches = re.findall("<link[^>]+?css.+?>", html_src)

In [9]:
# matches = re.findall("<style[^>]+css[^>]+>.+?</style>", html_src, re.DOTALL)

In [10]:
for idx, match in enumerate(matches):
    print(idx+1, ": ", match)

1 :  <link rel="stylesheet" href="/css/fonts.min.css">
2 :  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/KaTeX/0.8.3/katex.min.css" crossorigin="anonymous">
3 :  <link href="/css/13.9b4dc.css" rel="stylesheet">
4 :  <link href="/css/app.7100f.css" rel="stylesheet">
5 :  <link rel="stylesheet" type="text/css" href="/css/0.63cd2.css">
6 :  <link rel="stylesheet" type="text/css" href="/css/2.a44a1.css">
7 :  <link rel="stylesheet" type="text/css" href="/css/16.ca3a9.css">
8 :  <link rel="stylesheet" type="text/css" href="/css/1.99c19.css">
9 :  <link rel="stylesheet" type="text/css" href="/css/10.b8d63.css">
10 :  <link rel="stylesheet" type="text/css" href="/css/12.1e7dd.css">


Get both inline css style definition and links to the css style sheet files

In [11]:
soup = BeautifulSoup(driver.page_source)
css_link_elms = soup.find_all('link', {'rel': 'stylesheet'})
css_style_elms = soup.find_all('style', {'type':'text/css'})

In [12]:
len(css_link_elms)

10

In [13]:
import requests
import os
import urllib
from urllib.parse import urlparse

In [14]:
css_links = []
for idx, css_link_elm in enumerate(css_link_elms, start = 1):
    link = css_link_elm['href']
    if link.startswith('/'):
        link = urllib.parse.urljoin(base, link)
    css_links.append(link)
    print(idx, ": ", link)

1 :  https://classroom.udacity.com/css/fonts.min.css
2 :  https://cdnjs.cloudflare.com/ajax/libs/KaTeX/0.8.3/katex.min.css
3 :  https://classroom.udacity.com/css/13.9b4dc.css
4 :  https://classroom.udacity.com/css/app.7100f.css
5 :  https://classroom.udacity.com/css/0.63cd2.css
6 :  https://classroom.udacity.com/css/2.a44a1.css
7 :  https://classroom.udacity.com/css/16.ca3a9.css
8 :  https://classroom.udacity.com/css/1.99c19.css
9 :  https://classroom.udacity.com/css/10.b8d63.css
10 :  https://classroom.udacity.com/css/12.1e7dd.css


In [15]:
for css_link in css_links:
    r = requests.get(css_link, allow_redirects=True)
    a = urlparse(css_link)
    file_name = os.path.basename(a.path)                
    print("file_name= ", file_name) 
    open(file_name, 'wb').write(r.content)

file_name=  fonts.min.css
file_name=  katex.min.css
file_name=  13.9b4dc.css
file_name=  app.7100f.css
file_name=  0.63cd2.css
file_name=  2.a44a1.css
file_name=  16.ca3a9.css
file_name=  1.99c19.css
file_name=  10.b8d63.css
file_name=  12.1e7dd.css
